In [2]:
import pandas as pd
import numpy as np 
from bed_reader import open_bed, sample_file
import time
import re
import os
import math 
import itertools as itertools
from itertools import product
import statsmodels.stats.multitest
from scipy.stats import norm
from scipy.sparse import lil_matrix
from scipy.io import savemat, loadmat
from scipy.linalg import inv
# Using R inside python
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.vectors import FloatVector
# Set working directory
os.chdir("E:\\GWAS\\plink_win64_20231211")


### Block Sizes are determined manually so that each block contains ~3500 to ~4000 SNPs, you need to do it on your own for your data 
partition = np.array([3850,4100,3875,4050,3950,3950,4100,3710,3900,3750,3700,4200,4500,4450,4150,3800,4450,3900,3900,3500,3850,4400]); 
for chrno in range(1,23):
    num_snps_block = partition[chrno-1];
    print(num_snps_block);
    # chrno = 18;num_snps_block = 3870;
    filenamebed = f'ImpQcLDpCh{chrno}.bed'
    filenamebim = f'ImpQcLDpCh{chrno}.bim'
    filenamefam = f'ImpQcLDpCh{chrno}.fam'
    
    import pandas as pd
    import numpy as np
    # phen = pd.read_csv("height&id.csv")
    # phen = phen.iloc[:,[1,2]]
    # phenidnonmiss = phen.drop(np.where(np.array(np.transpose(phen.iloc[:,1])).astype('str') == 'nan')[0])
    from bed_reader import open_bed
    with open_bed(filenamebed) as bed3:
        idd = bed3.iid[:];
        snp = bed3.sid[:];
        print(snp.shape)
        print(idd.shape)
    
    with open_bed(filenamebed) as bed:
        idd = bed.iid[:];
        snp = bed.sid[:];
    phen = pd.read_csv("height&id.csv")
    phen = phen.iloc[:,[1,2]]
    phenidnonmiss = phen.drop(np.where(np.array(np.transpose(phen.iloc[:,1])).astype('str') == 'nan')[0])
    common_iid = np.intersect1d(idd,phenidnonmiss['eid'])
    common_iid = common_iid.astype('int64')
    order_common_iid = idd[np.where(np.isin(idd,common_iid)==True)[0]]
    argsort = np.argsort(order_common_iid.astype('int64'))
    phen_filt = phenidnonmiss.loc[phenidnonmiss['eid'].isin(common_iid)];
    height_unordered = phen_filt['50-0.0'] #21001-0.0
    # height_unordered = phen_filt['21001-0.0']
    height = np.array(height_unordered)#[argsort]
    # comn_snps = pd.read_csv("index_common.csv")
    
    bim20 = open(filenamebim,"r");
    bim20 = bim20.readlines();
    func = lambda x: re.split(r'\t+',x[:-1])
    x_func = np.vectorize(func)
    bim20 = np.array([np.concatenate([x_func(x)]) for x in bim20]);
    # bim20
    
    LDbreakbed = f'fourier_ls-chr{chrno}.bed'
    indblock20 = open(LDbreakbed,"r");
    indblock20 = indblock20.readlines();
    func = lambda x: re.split(r'\t+',x[:-1])
    x_func = np.vectorize(func)
    indblock20 = np.array([np.concatenate([x_func(x)]) for x in indblock20]);
    # indblock20
    
    rsidlist = [];
    for indx in indblock20[1:]:
        print(indx);
        idx = np.arange(indx[1].astype('int'),indx[2].astype('int')).astype('<U10');
        rsid = bim20[pd.DataFrame(bim20)[3].isin(idx)][:,1];
        rsidlist.append(rsid);
    
    num_snps = 0;
    for rsid in rsidlist:
        print(np.size(rsid));
        num_snps+=np.size(rsid);
    print(num_snps)
    
    rsidlistn = [];                    
    rsidn = np.array([]);
    for rsid in rsidlist:
        if np.size(rsidn)<num_snps_block:
            rsidn = np.append(rsidn,rsid);
        else:
            rsidlistn.append(rsidn); 
            rsidn = np.array([]);
            rsidn = np.append(rsidn,rsid);
    rsidlistn.append(rsidn);
    for rsidn in rsidlistn:
        print(np.size(rsidn));
    
    from scipy.sparse import lil_matrix
    y=height;
    Z = [];
    ScovJ = [];
    Rsid = [];
    step_size = 1;
    with open_bed(filenamebed) as bed20:
        for rsid in rsidlistn:
            print("done");
            # K = np.size(rsid)
            # X=bed20.read(index=np.s_[np.where(np.isin(idd,common_iid)==True)[0][argsort],np.isin(bed20.sid,rsid)]);
            # X=bed20.read(index=np.s_[np.where(np.isin(idd,common_iid)==True)[0][argsort],
            #              np.where(np.isin(bed20.sid,rsid)==True)[0]]);
            jump_index = np.arange(int(np.size(rsid)/step_size))*step_size;
            filtered_snps = np.where(np.isin(bed20.sid,rsid)==True)[0][jump_index];
            X=bed20.read(index=np.s_[np.where(np.isin(idd,common_iid)==True)[0][argsort],filtered_snps]);
            # X = 2-X;
            print(X.shape);
            snp = rsid[jump_index];
            X = pd.DataFrame(X);
            # prop_nan = np.divide((X.isna().sum()),X.shape[0])
            # idx3 = np.where(prop_nan<0.1)[0]
            # X = X.iloc[:, idx3]
            # snp_id = snp[idx3]
            # X = X.fillna(X.mean());
            idx1 = np.where(X.isna().any() == False)[0];
            X = X.iloc[:,idx1];
            snp_id = snp[idx1];
            idx2 = (X != 0).any(axis=0);
            X = X.loc[:, idx2]
            snp_id = snp_id[idx2]
            # print(X)
            # print(snp_id)
            X = (X - np.mean(X)) #/ np.std(X)
            y = (y-np.mean(y))#/np.std(y)
            XTX = np.matmul(np.transpose(np.array(X)),np.array(X));
            if np.any(np.linalg.eigvals(XTX) <= 1e-10):
                print("not pd")
                continue;
            XTXinv = np.linalg.inv(XTX);
            XTy = np.matmul(np.transpose(np.array(X)),np.array(y));
            betahat = np.matmul(XTXinv,XTy)
            e =  y-np.matmul(np.array(X),betahat)
            est_sigma = np.sqrt(np.dot(e,e)/(np.shape(X)[0]-np.shape(X)[1]))
            # S_inv = np.diag(np.sqrt(1./np.diag(np.array(XTXinv))));
            S_inv = np.diag((np.diag(XTXinv)**-0.5));
            corrzscore = (np.matmul(np.matmul(S_inv,XTXinv),S_inv));
            del XTXinv
            del XTX
            del e
            del XTy
            del X;
            zscore = np.matmul(S_inv,betahat)/est_sigma
            del(S_inv);
            del(betahat);
            Z.append(zscore);
            K = np.size(zscore);
            N=5;
            scovJ = lil_matrix((K, K));
            for n in range((2*N)+1):
                scovJ.setdiag(corrzscore.diagonal(n),n);
                scovJ.setdiag(corrzscore.diagonal(n),-n);
            del(corrzscore);                                                                                                                                                                                                                                                
            ScovJ.append(scovJ);
            del(scovJ);
            Rsid.append(snp_id);
            del(snp_id);
        
    ScovJ_dict = {f'matrix_{idx}': matrix for idx, matrix in enumerate(ScovJ)}
    savemat(f'ScovJp_chr{chrno}.mat', ScovJ_dict)
    np.savez(f'zJp_chr{chrno}.npz', *Z)
    np.savez(f'rsidlistp_chr{chrno}.npz', *Rsid)

3850
(8221,)
(487409,)
['chr21 ' ' 9411243 ' ' 15950982']
['chr21 ' ' 15950982 ' ' 18053165']
['chr21 ' ' 18053165 ' ' 19480375']
['chr21 ' ' 19480375 ' ' 20962339']
['chr21 ' ' 20962339 ' ' 22072764']
['chr21 ' ' 22072764 ' ' 23362875']
['chr21 ' ' 23362875 ' ' 24696231']
['chr21 ' ' 24696231 ' ' 26469122']
['chr21 ' ' 26469122 ' ' 27271019']
['chr21 ' ' 27271019 ' ' 29125226']
['chr21 ' ' 29125226 ' ' 31197025']
['chr21 ' ' 31197025 ' ' 32668642']
['chr21 ' ' 32668642 ' ' 34376999']
['chr21 ' ' 34376999 ' ' 35225465']
['chr21 ' ' 35225465 ' ' 36524373']
['chr21 ' ' 36524373 ' ' 37870779']
['chr21 ' ' 37870779 ' ' 38711704']
['chr21 ' ' 38711704 ' ' 40482902']
['chr21 ' ' 40482902 ' ' 41389527']
['chr21 ' ' 41389527 ' ' 43321528']
['chr21 ' ' 43321528 ' ' 44506268']
['chr21 ' ' 44506268 ' ' 46177105']
['chr21 ' ' 46177105 ' ' 47492226']
['chr21 ' ' 47492226 ' ' 48119752']
209
500
322
364
249
289
235
354
124
460
288
341
457
226
351
302
220
434
244
702
461
506
395
188
8221
4192
4029
don